# Hypsometric Map of South Africa (WorldClim DEM + Esri Satellite)

In this notebook we build a **hypsometric map** of South Africa using:

- a global elevation raster from **WorldClim** (DEM),
- a national boundary for South Africa,
- an **Esri World Imagery** satellite basemap,
- and the Python libraries **GeoPandas**, **rioxarray**, and **Folium**.

The main steps are:

1. Load the elevation raster and South Africa boundary.
2. Make sure both layers use the same **Coordinate Reference System (CRS)**.
3. **Clip the raster to the country boundary** (so we only see South Africa).
4. Downsample the raster to make it lighter for web display.
5. Turn the raster into a coloured RGB image using a terrain colour scale.
6. Create an interactive Folium map with:
   - Esri satellite as the basemap,
   - the coloured elevation raster on top,
   - and the South African boundary as a line overlay.
7. Save the map as **HTML** and show it inside the notebook.

## Data sources and links

Below are sources used for this notebook.  

- **Administrative boundaries (country + provinces)**
  - SimpleMaps – free South Africa admin boundaries (Shapefile / GeoJSON)  
    - https://simplemaps.com/gis/country/za

- **Elevation data**
  - WorldClim - Historical climate data
    - https://www.worldclim.org/data/worldclim21.html

## Import Python libraries

We first import all the libraries we will use:

- `Path` from `pathlib` for clean and portable file paths.
- `geopandas` to read and work with vector data (the South Africa boundary).
- `rioxarray` to read and manipulate raster data (the DEM).
- `numpy` for numerical operations on arrays.
- `folium` for interactive web maps.
- `branca.colormap` to build a legend for the elevation colours.
- `matplotlib.cm` and `matplotlib.colors` to build a colour ramp and convert colours to hex strings.
- `IFrame` from `IPython.display` to display an external HTML map inside the notebook.

If you are missing any of these packages, you can install them with by uncommenting:

In [2]:
# import python libraries

from pathlib import Path

import geopandas as gpd
import rioxarray as rxr
import numpy as np
import folium
import branca.colormap as bcm
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from IPython.display import IFrame

## Set up the data folder and file paths

We will assume the data for this section lives in a folder called `resources/02_hypsometric` inside our project.

In that folder we expect two files:

- `wc2.1_30s_elev.tif` – a **WorldClim** 30 arc-second elevation raster (global DEM).
- `south_africa_whole.geojson` – a polygon that outlines South Africa (renamed).

Here we create `Path` objects to these files. This cell does **not** download data; it only tells Python where the files are.

In [3]:
# Define the data folder for this section
data_folder = Path("resources/02_hypsometric")
data_folder.mkdir(exist_ok=True)

# Paths to the elevation raster and national boundary
worldclim_elevation_path = data_folder / "wc2.1_30s_elev.tif"
south_africa_boundary_path = data_folder / "south_africa_whole.geojson"

worldclim_elevation_path, south_africa_boundary_path

(WindowsPath('resources/02_hypsometric/wc2.1_30s_elev.tif'),
 WindowsPath('resources/02_hypsometric/south_africa_whole.geojson'))

## Load the South Africa boundary and elevation raster

We now load:

- the South African boundary as a **GeoDataFrame** using GeoPandas, and
- the WorldClim elevation raster as an **xarray DataArray** using `rioxarray.open_rasterio`.

We pass `masked=True` so that missing values are handled cleanly (they will be stored as `NaN`).

In [4]:
# Load South Africa boundary
south_africa_boundary = gpd.read_file(south_africa_boundary_path)
south_africa_boundary

,source,id,name,geometry
0,https://simplemaps.com,ZA,South Africa,"MULTIPOLYGON (((19.98145 -24.75249, 19.98641 -..."


In [5]:
# Load the WorldClim elevation raster
south_africa_elevation_worldclim = rxr.open_rasterio(worldclim_elevation_path, masked=True)
south_africa_elevation_worldclim

<xarray.DataArray (band: 1, y: 21600, x: 43200)> Size: 4GB
[933120000 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * y            (y) float64 173kB 90.0 89.99 89.98 ... -89.98 -89.99 -90.0
  * x            (x) float64 346kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  8424
    STATISTICS_MEAN:     1.#SNAN
    STATISTICS_MINIMUM:  -415
    STATISTICS_STDDEV:   1.#SNAN
    scale_factor:        1.0
    add_offset:          0.0

## Check and align the Coordinate Reference Systems (CRS)

A **Coordinate Reference System (CRS)** describes how the coordinates in our data relate to real positions on Earth.

For the overlay to look correct:

- the raster and the boundary must use the **same CRS** (same way of describing latitude and longitude).

In this step we:

1. Check the CRS of the boundary (`south_africa_boundary.crs`).
2. Check the CRS of the raster (`south_africa_elevation_worldclim.rio.crs`).
3. If the raster has no CRS defined, we write one. WorldClim 30 arc-second data are normally in **WGS 84 (EPSG:4326)**.
4. Reproject the boundary to match the raster CRS if needed.

This alignment is what keeps the boundary well lined up with both the DEM and the Esri basemap.

In [6]:
# Check the CRS of the boundary and raster
boundary_crs = south_africa_boundary.crs
raster_crs = south_africa_elevation_worldclim.rio.crs

print("Boundary CRS:", boundary_crs)
print("Raster CRS:", raster_crs)

Boundary CRS: EPSG:4326
Raster CRS: EPSG:4326


In [7]:
# If the raster CRS is missing, set it explicitly to WGS84 (EPSG:4326)
if south_africa_elevation_worldclim.rio.crs is None:
    south_africa_elevation_worldclim = south_africa_elevation_worldclim.rio.write_crs("EPSG:4326")

# Make sure the boundary uses the same CRS as the raster
south_africa_boundary = south_africa_boundary.to_crs(south_africa_elevation_worldclim.rio.crs)

south_africa_boundary.crs, south_africa_elevation_worldclim.rio.crs

(<Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - undefined
 Datum: World Geodetic System 1984
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich,
 CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'))

## Select the elevation band

The WorldClim elevation file is read as a raster with a `band` dimension. In this case, there is usually only **one band** that gives elevation in metres.

We extract that single band as `south_africa_elevation_single` so we can work with it more easily.

In [8]:
# Extract the first (and usually only) band
south_africa_elevation_single = south_africa_elevation_worldclim.isel(band=0)
south_africa_elevation_single

<xarray.DataArray (y: 21600, x: 43200)> Size: 4GB
[933120000 values with dtype=float32]
Coordinates:
  * y            (y) float64 173kB 90.0 89.99 89.98 ... -89.98 -89.99 -90.0
  * x            (x) float64 346kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
    band         int64 8B 1
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  8424
    STATISTICS_MEAN:     1.#SNAN
    STATISTICS_MINIMUM:  -415
    STATISTICS_STDDEV:   1.#SNAN
    scale_factor:        1.0
    add_offset:          0.0

## Clip the raster to the South Africa boundary

The WorldClim DEM is global, but we only want the part **inside South Africa**.

We use `rio.clip` from `rioxarray`:

- `south_africa_boundary.geometry` gives the polygon(s) that define South Africa.
- `south_africa_boundary.crs` tells `rioxarray` which CRS the geometry uses.
- `drop=True` removes data outside the polygons completely.

The result is a new raster that covers only South Africa.

In [10]:
# Get the boundary geometries
south_africa_geometries = south_africa_boundary.geometry

# Clip the elevation raster to the South Africa boundary
south_africa_elevation_clipped = south_africa_elevation_single.rio.clip(
    south_africa_geometries,
    south_africa_boundary.crs,
    drop=True
)

south_africa_elevation_clipped

MemoryError: Unable to allocate 890. MiB for an array with shape (21600, 43200) and data type bool

## Check dimension names and downsample the raster

Raster data in xarray can use different names for the horizontal axes:

- Some rasters use `x` and `y`.
- Others use `lon` (longitude) and `lat` (latitude).

We write a small check to detect which names our raster uses so our later code stays flexible.

### Why downsample?

Web maps do not need the full original resolution of the DEM. To make the map lighter and faster:

- we **coarsen** the raster by a factor (for example 4),
- this averages blocks of 4×4 pixels into one pixel,
- the overall shape of elevation is still clear, but there are fewer pixels to draw.

We store this downsampled raster as `south_africa_elevation_coarse`.

In [ ]:
# Detect the horizontal dimension names (x/y or lon/lat)
if "x" in south_africa_elevation_clipped.dims and "y" in south_africa_elevation_clipped.dims:
    x_dim_name = "x"
    y_dim_name = "y"
elif "lon" in south_africa_elevation_clipped.dims and "lat" in south_africa_elevation_clipped.dims:
    x_dim_name = "lon"
    y_dim_name = "lat"
else:
    raise ValueError(f"Unexpected dimension names: {south_africa_elevation_clipped.dims}")

x_dim_name, y_dim_name

In [ ]:
# Downsample (coarsen) the clipped raster to make it lighter for web display
downsample_factor = 4  # you can try 3, 4, 5, etc.

south_africa_elevation_coarse = south_africa_elevation_clipped.coarsen(
    {x_dim_name: downsample_factor, y_dim_name: downsample_factor},
    boundary="trim"
).mean()

south_africa_elevation_coarse

## Compute map bounds and turn the DEM into an RGB image

Folium's `ImageOverlay` expects:

- an image (2D RGB array) and
- geographic **bounds** given as `[[south, west], [north, east]]` in latitude and longitude.

We therefore:

1. Use `rio.bounds()` to get the bounding box of our coarse DEM:
   - `west`, `south`, `east`, `north`.
2. Extract the DEM values as a NumPy array.
3. Find the minimum and maximum elevation (ignoring missing values).
4. Normalize the values to the range 0–1.
5. Feed the normalized values through a matplotlib colour map (here `"terrain"`), which gives us RGBA colours.
6. Keep only the RGB channels and scale them to 0–255 as `uint8` integers.

This RGB image will be draped over the map inside the South African boundary.

In [ ]:
# Get bounds of the coarse DEM (west, south, east, north)
west, south, east, north = south_africa_elevation_coarse.rio.bounds()
west, south, east, north

In [ ]:
# Convert the DEM to a NumPy array
dem_array = south_africa_elevation_coarse.values

# Compute min and max elevation (ignoring NaNs)
dem_min = float(np.nanmin(dem_array))
dem_max = float(np.nanmax(dem_array))

dem_min, dem_max

In [ ]:
# Normalise the DEM values to 0–1
normalized = (dem_array - dem_min) / (dem_max - dem_min)
normalized = np.clip(normalized, 0, 1)  # safety guard

# Use a matplotlib colormap (terrain) to get RGBA colours
mpl_cmap = cm.get_cmap("terrain")
rgba = mpl_cmap(normalized)  # shape: (rows, cols, 4)

# Keep only the RGB channels and scale to 0–255
rgb_image = (rgba[:, :, :3] * 255).astype("uint8")

rgb_image.shape

## Build a matching legend with `branca.colormap`

We now create a `LinearColormap` from `branca` for the legend. To keep the legend colours aligned with the raster:

- we sample several points from the same matplotlib colour map (`"terrain"`),
- convert those colours to hex strings,
- and use them in the `LinearColormap`.

We also set:

- `vmin` and `vmax` to the same `dem_min` and `dem_max` as the raster,
- and add a caption `"Elevation (m)"`.

In [ ]:
# Build a list of hex colours sampled from the same matplotlib colormap
hex_colors = [mcolors.to_hex(mpl_cmap(v)) for v in np.linspace(0, 1, 7)]

# Create a branca LinearColormap for the legend
elevation_colormap = bcm.LinearColormap(
    colors=hex_colors,
    vmin=dem_min,
    vmax=dem_max
)
elevation_colormap.caption = "Elevation (m)"

elevation_colormap

## Create the Folium map with Esri satellite and overlays

We now assemble the interactive map.

1. **Base map**: Esri World Imagery satellite tiles as an XYZ layer.
2. **DEM overlay**: our RGB elevation image, using the bounds `(south, west, north, east)`.
3. **Boundary overlay**: the South Africa polygon as a transparent shape with a black outline.
4. **Legend**: the elevation colour bar added to the map.
5. **Layer control**: a small control in the top-right to toggle layers.

Because we carefully aligned the CRS and used the clipped DEM, the boundary and the elevation should sit correctly on top of the satellite basemap.

In [ ]:
# Esri World Imagery XYZ URL
ESRI_WORLD_IMAGERY = (
    "https://services.arcgisonline.com/ArcGIS/rest/services/"
    "World_Imagery/MapServer/tile/{z}/{y}/{x}"
)

# Use the centroid of the boundary as the map centre
boundary_union = south_africa_boundary.unary_union
boundary_centroid = boundary_union.centroid
center_lat, center_lon = boundary_centroid.y, boundary_centroid.x

center_lat, center_lon

In [ ]:
# Create the folium map
south_africa_hypsometric_map = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=5,
    tiles=None  # we add our own satellite basemap
)

# Add Esri satellite basemap
folium.TileLayer(
    tiles=ESRI_WORLD_IMAGERY,
    attr="Esri World Imagery",
    name="Esri Satellite",
    overlay=False,
    control=True,
).add_to(south_africa_hypsometric_map)

# Add the DEM as an image overlay (note the order: [south, west], [north, east])
folium.raster_layers.ImageOverlay(
    image=rgb_image,
    bounds=[[south, west], [north, east]],
    opacity=0.80,
    name="WorldClim elevation (30 arc-second)",
).add_to(south_africa_hypsometric_map)

# Add the South Africa boundary as a line overlay
folium.GeoJson(
    south_africa_boundary,
    name="South Africa boundary",
    style_function=lambda feature: {
        "fillOpacity": 0.0,
        "color": "black",
        "weight": 1.5,
    },
).add_to(south_africa_hypsometric_map)

# Add the elevation legend and layer control
elevation_colormap.add_to(south_africa_hypsometric_map)
folium.LayerControl().add_to(south_africa_hypsometric_map)

south_africa_hypsometric_map